In [1]:
import requests
page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
page.status_code

200

In [3]:
# We can print out the HTML content of the page using the content property:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

## Parsing a page with BeautifulSoup

In [4]:
# As you can see above, we now have downloaded an HTML document.
# We can use the BeautifulSoup library to parse this document, and extract the text from the p tag.
#We first have to import the library, and create an instance of the BeautifulSoup class to parse our document:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
# We can now print out the HTML content of the page, formatted nicely, using the prettify method on the BeautifulSoup object.
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


As all the tags are nested, we can move through the structure one level at a time. We can first select all the elements at the top level of the page using the children property of soup.

In [6]:
# Note that children returns a list generator, so we need to call the list function on it:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

The above tells us that there are two tags at the top level of the page — the initial <!DOCTYPE html> tag, and the <html> tag. There is a newline character (n) in the list as well

In [7]:
# Let’s see what the type of each element in the list is:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

As we can see, all of the items are BeautifulSoup objects:

1. The first is a Doctype object, which contains information about the type of the document.

2. The second is a NavigableString, which represents text found in the HTML document.

3. The final item is a Tag object, which contains other nested tags.

The most important object type, and the one we’ll deal with most often, is the Tag object.


The Tag object allows us to navigate through an HTML document, and extract other tags and text. You can learn more about the various BeautifulSoup objects

In [8]:
# We can now select the html tag and its children by taking the third item in the list:
html = list(soup.children)[2]

Each item in the list returned by the children property is also a BeautifulSoup object, so we can also call the children method on html.

In [9]:
# Now, we can find the children inside the html tag:
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

As we can see above, there are two tags here, head, and body. We want to extract the text inside the p tag, so we’ll dive into the body:

In [10]:
body = list(html.children)[3]

In [11]:
# Now, we can get the p tag by finding the children of the body tag:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [12]:
# We can now isolate the p tag:
p = list(body.children)[1]

In [13]:
# Once we’ve isolated the tag, we can use the get_text method to extract all of the text inside the tag:
p.get_text()

'Here is some simple content for this page.'

## Finding all instances of a tag at once

What we did above was useful for figuring out how to navigate a page, but it took a lot of commands to do something fairly simple.
If we want to extract a single tag, we can instead use the find_all method, which will find all the instances of a tag on a page.

In [14]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [15]:
# Note that find_all returns a list, so we’ll have to loop through, or use list indexing, it to extract text:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [16]:
# If you instead only want to find the first instance of a tag, you can use the find method, which will return a single
# BeautifulSoup object:
soup.find('p')

<p>Here is some simple content for this page.</p>

## Searching for tags by class and id

We introduced classes and ids earlier, but it probably wasn’t clear why they were useful.

Classes and ids are used by CSS to determine which HTML elements to apply certain styles to. But when we’re scraping, we can also use them to specify the elements we want to scrape.

To illustrate this principle, we’ll work with the following page:
    
We can access the particular document at the URL https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html.

Let’s first download the page and create a BeautifulSoup object:

In [17]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

Now, we can use the find_all method to search for items by class or by id. In the below example, we’ll search for any p tag that has the class outer-text:

In [18]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [19]:
# In the below example, we’ll look for any tag that has the class outer-text:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [20]:
# We can also search for elements by id:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

## Time to Start Scraping!

We now know enough to download the page and start parsing it. In the below code, we will:

1. Download the web page containing the forecast.

2. Create a BeautifulSoup class to parse the page.

3. Find the div with id seven-day-forecast, and assign to seven_day

4. Inside seven_day, find each individual forecast item.

5. Extract and print the first forecast item

In [21]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  NOW until
  <br/>
  6:00am Wed
 </p>
 <p>
  <img alt="" class="forecast-icon" src="newimages/medium/novc.png" title=""/>
 </p>
 <p class="short-desc">
  High Surf Advisory
 </p>
</div>


## Extracting information from the page

As we can see, inside the forecast item tonight is all the information we want. There are four pieces of information we can extract:

1. The name of the forecast item — in this case, Tonight.

2. The description of the conditions — this is stored in the title property of img.

3. A short description of the conditions — in this case, Mostly Clear.

4. The temperature low — in this case, 49 degrees.

We’ll extract the name of the forecast item, the short description, and the temperature first, since they’re all similar:

In [31]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
#temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
#print(temp)

NOW until6:00am Wed
High Surf Advisory


In [34]:
# Now, we can extract the title attribute from the img tag. To do this, we just treat the BeautifulSoup object like a 
# dictionary, and pass in the attribute we want as a key:
img = tonight.find("img")
desc = img['title']
#print(desc)
print(img)

<img alt="" class="forecast-icon" src="newimages/medium/novc.png" title=""/>


## Extracting all the information from the page

In [35]:
# Select all items with the class period-name inside an item with the class tombstone-container in seven_day.
# Use a list comprehension to call the get_text method on each BeautifulSoup object.
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['NOW until6:00am Wed',
 'Overnight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday']

As we can see above, our technique gets us each of the period names, in order.

We can apply the same technique to get the other three fields:

In [38]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['High Surf Advisory', 'Cloudy', 'Patchy Fogthen MostlyCloudy', 'Patchy Fog', 'Areas Fogthen MostlyCloudy', 'Slight ChanceRain', 'Rain Likely', 'Partly Cloudy', 'Mostly Sunny']
['Low: 50 °F', 'High: 58 °F', 'Low: 50 °F', 'High: 57 °F', 'Low: 49 °F', 'High: 57 °F', 'Low: 45 °F', 'High: 56 °F']
['', 'Overnight: Cloudy, with a low around 50. Calm wind. ', 'Wednesday: Patchy fog between 11am and noon.  Otherwise, mostly cloudy, with a high near 58. South wind around 5 mph becoming calm  in the morning. ', 'Wednesday Night: Patchy fog after 9pm.  Otherwise, mostly cloudy, with a low around 50. Calm wind. ', 'Thursday: Areas of fog before 11am.  Otherwise, mostly cloudy, with a high near 57. Calm wind. ', 'Thursday Night: A 20 percent chance of rain after 10pm.  Mostly cloudy, with a low around 49. Light southwest wind.  New precipitation amounts of less than a tenth of an inch possible. ', 'Friday: Rain likely.  Mostly cloudy, with a high near 57. Chance of precipitation is 70%. New precipi

## Combining our data into a Pandas Dataframe

We can now combine the data into a Pandas DataFrame and analyze it. A DataFrame is an object that can store tabular data, making data analysis easy. If you want to learn more about Pandas, check out our free to start course here.

In order to do this, we’ll call the DataFrame class, and pass in each list of items that we have. We pass them in as part of a dictionary.

Each dictionary key will become a column in the DataFrame, and each list will become the values in the column:

In [40]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "desc":descs
})
weather

,period,short_desc,desc
0,NOW until6:00am Wed,High Surf Advisory,
1,Overnight,Cloudy,"Overnight: Cloudy, with a low around 50. Calm ..."
2,Wednesday,Patchy Fogthen MostlyCloudy,Wednesday: Patchy fog between 11am and noon. ...
3,WednesdayNight,Patchy Fog,Wednesday Night: Patchy fog after 9pm. Otherw...
4,Thursday,Areas Fogthen MostlyCloudy,Thursday: Areas of fog before 11am. Otherwise...
5,ThursdayNight,Slight ChanceRain,Thursday Night: A 20 percent chance of rain af...
6,Friday,Rain Likely,"Friday: Rain likely. Mostly cloudy, with a hi..."
7,FridayNight,Partly Cloudy,"Friday Night: Partly cloudy, with a low around..."
8,Saturday,Mostly Sunny,"Saturday: Mostly sunny, with a high near 56."
